<a href="https://colab.research.google.com/github/slon101/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [5]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  
  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks= [tensorboard_callback]
  )
  return model 

def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, padding='same'),
  Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
  Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3), 
  
  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax')
  ])

In [10]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 8s 30ms/step - loss: 2.4453 - accuracy: 0.3210 - val_loss: 0.6478 - val_accuracy: 0.8032
Epoch 2/5
272/272 [==============================] - 7s 27ms/step - loss: 0.5164 - accuracy: 0.8381 - val_loss: 0.1643 - val_accuracy: 0.9502
Epoch 3/5
272/272 [==============================] - 8s 28ms/step - loss: 0.2320 - accuracy: 0.9298 - val_loss: 0.0677 - val_accuracy: 0.9825
Epoch 4/5
272/272 [==============================] - 8s 28ms/step - loss: 0.1510 - accuracy: 0.9556 - val_loss: 0.0392 - val_accuracy: 0.9886
Epoch 5/5
272/272 [==============================] - 8s 28ms/step - loss: 0.1288 - accuracy: 0.9614 - val_loss: 0.0233 - val_accuracy: 0.9931


0.9784580498866213

In [11]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 1s 4ms/step - loss: 0.0866 - accuracy: 0.9785


[0.08664946258068085, 0.978458046913147]

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, padding='same'),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),
    
    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax')
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  
  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [0]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10), 
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7)
}

In [15]:
best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 120.0, 'dropout_cnn_block_one': 0.49529607306546164, 'dropout_cnn_block_three': 0.39718112357313873, 'dropout_cnn_block_two': 0.4135102947598675, 'dropout_dense_block_one': 0.5848748575860789, 'dropout_dense_block_two': 0.47170532528181464}
accuracy=0.8936507701873779
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.3650661315428566, 'dropout_cnn_block_three': 0.4206515712709814, 'dropout_cnn_block_two': 0.4673480942431087, 'dropout_dense_block_one': 0.5870929568992638, 'dropout_dense_block_two': 0.44777655595178545}
accuracy=0.9668934345245361
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.34643098798122507, 'dropout_cnn_block_three': 0.40953720948268146, 'dropout_cnn_block_two': 0.38328555970342976, 'dropout_dense_block_one': 0.5712837568263254, 'dropout_dense_block_two': 0.469447998105316}
accuracy=0.9727891087532043
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.38065106320544484, 'dropout_cnn_block_three': 0.48631496485046094, 'dropout_cnn_block_two': 0.38064042